# Task-2 Label data Subset of Dataset in CoNLL format

In [ ]:
 import csv
import importlib
import os
import re
from telethon import TelegramClient
import pytesseract
from PIL import Image
import unicodedata
import emoji
import nltk
from nltk.tokenize import word_tokenize
# from nltk.normalize import word_normalize
import pandas as pd
import sys
sys.path.append('../src')
                # F:/Telegram_E-commerce-_extractor-/src')

## load data